In [4]:
from google.colab import drive
import pandas as pd

# Esto montará tu Drive en el cuaderno
drive.mount('/content/drive')

# Aquí pegarías la lógica que usamos para procesar
# Y al final guardarías así:
# df_final.to_csv('/content/drive/My Drive/loans_processed_flattened.csv', index=False)

import pandas as pd
import ast

# 1. Cargar el archivo original
# IMPORTANT: Please upload your 'AE_challenge_loans_ori_conv_1.csv' file to Colab
# or update the filename below to the correct path/name of your file.
# For demonstration purposes, I'm replacing it with a placeholder that will likely also fail,
# but allows the code to be structurally correct.
df = pd.read_csv('/content/drive/My Drive/AE_challenge_loans.csv') # Ensure this file exists and is correctly named and located.

# 2. Función para convertir el string de la columna en una lista real de diccionarios
def parse_schedule(x):
    try:
        # Usamos ast.literal_eval para manejar las comillas del CSV
        return ast.literal_eval(x)
    except Exception:
        return []

# Aplicamos la conversión a la columna original_schedule
df['original_schedule'] = df['original_schedule'].apply(parse_schedule)

# 3. "Explotar" (explode) la columna para crear una fila por cada elemento de la lista
df_exploded = df.explode('original_schedule').reset_index(drop=True)

# 4. Expandir los diccionarios en columnas nuevas (due_date, payment_due, etc.)
# pd.json_normalize toma una serie de diccionarios y los convierte en un DataFrame
schedule_columns = pd.json_normalize(df_exploded['original_schedule'])

# 5. Combinar loan_id y user_id con las nuevas columnas de pagos
final_df = pd.concat([df_exploded[['loan_id', 'user_id']], schedule_columns], axis=1)

# Mostrar las primeras 10 filas para verificar
print(final_df.head(10).to_markdown(index=False))

# Guardar a un nuevo CSV
final_df.to_csv('/content/drive/My Drive/loans_processed_flattened.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
|             loan_id | user_id                  | due_date   |   payment_due |   interest_due |   principal_due |
|--------------------:|:-------------------------|:-----------|--------------:|---------------:|----------------:|
| 1599101102672337175 | 67890bcc531b1ced29f50642 | 2025-03-18 |         127   |              0 |           127   |
| 1599101102672337175 | 67890bcc531b1ced29f50642 | 2025-03-31 |         126.1 |              0 |           126.1 |
| 1599101102672337175 | 67890bcc531b1ced29f50642 | 2025-03-18 |         127   |              0 |           127   |
| 1599101102672337175 | 67890bcc531b1ced29f50642 | 2025-03-31 |         126.1 |              0 |           126.1 |
| 1599101102672337175 | 67890bcc531b1ced29f50642 | 2025-03-18 |         127   |              0 |           127   |
| 1599101102672337175 | 67890bcc531b1ced29f50642 | 2025-03-31 |   